In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    # add_connection
)
from buildingmotif.namespaces import BRICK, RDF, S223
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 's223'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey(f'{ontology}-test_site','test_build','.', overwrite=True, ontology=ontology,
                 template_dict = {'zone': 'hvac-zone',
                                  "space": "space",
                                  "hvac": "hp-rtu",
                                  "tstat": "tstat",
                                  "window": "window",
                                  "site": "site" })

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values names: {}
values: dict_values(['area-value'])
values names: {'area': '<name>-area'}
values: dict_values(['cooling_capacity-value', 'heating_capacity-value', 'cooling_COP-value', 'heating_COP-value'])
values names: {'cooling_capacity': '<name>-cooling_capacity', 'cooling_COP': '<name>-cooling_COP', 'heating_COP': '<name>-heating_COP', 'heating_capacity': '<name>-heating_capacity'}
values: dict_values(['setpoint_deadband-value', 'active-value', 'resolution-value', 'tolerance-value', 'stage_count-value'])
values names: {'active': '<name>-active', 'resolution': '<name>-resolution', 'setpoint_deadband': '<name>-setpoint_deadband', 'tolerance': '<name>-tolerance', 'stage_count': '<name>-stage_count'}
values: dict_values(['tilt-value', 'azimuth-value', 'area-value'])
values names: {'area': '<name>-area', 'tilt': '<name>-tilt', 'azimuth': '<name>-azim

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
prefill_csv_survey(f'{ontology}-test_site/test_build')

Loading config from: s223-test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 10 empty cells in zone.csv
No empty cells found in site.csv
No empty cells found in window.csv
No empty cells found in hvac.csv
No empty cells found in point_list.csv
No empty cells found in tstat.csv
No empty cells found in space.csv


In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 3)
New columns added: ['area']


Exception: Paramaters {'area'} are still unused!

# Testing SHACL Generation and Validation

In [ ]:
og = clone.clone_graph(s.graph)

In [ ]:
# Create handler
handler = SHACLHandler(ontology=ontology)

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Validate a model
conforms, results_graph, results_text = handler.validate_model(s.graph)

s.graph.serialize(f'{ontology}-test-model-reasoned.ttl', format = 'ttl')
if not conforms:
    print("Validation failed:")
    print(results_text)

In [ ]:
# lots of new inferred information
(s.graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/s223-test_site#hvac_1> ns1:air-connected-to <urn:hpflex/s223-test_site#zone_1> ;
    ns1:has-point <urn:hpflex/s223-test_site#1.0> .

<urn:hpflex/s223-test_site#hvac_2> ns1:air-connected-to <urn:hpflex/s223-test_site#zone_2> ;
    ns1:has-point <urn:hpflex/s223-test_site#2.0> .

<urn:hpflex/s223-test_site#hvac_3> ns1:air-connected-to <urn:hpflex/s223-test_site#zone_3> ;
    ns1:has-point <urn:hpflex/s223-test_site#3.0> .

<urn:hpflex/s223-test_site#s223-test_site> a ns1:space ;
    ns1:has-point <urn:hpflex/s223-test_site#1.0> .

<urn:hpflex/s223-test_site#tstat_zone_1> ns1:has-location <urn:hpflex/s223-test_site#zone_1> ;
    ns1:has-point <urn:hpflex/s223-test_site#1.0> .

<urn:hpflex/s223-test_site#tstat_zone_2> ns1:has-location <urn:hpflex/s223-test_site#zone_2> ;
    ns1:has-point <urn:hpflex/s223-test_site#2.0> .

<urn:hpflex/s223-test_site#tstat_zone_3> ns1:has-location <urn:hpflex/s223-test_site#zone_3> ;
    ns1:has-point <urn:h

# Testing get Metadata

In [ ]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel(f"{ontology}-test_site/test_build/test_build.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology)
site_info = loader.get_all_building_objects()

In [ ]:
print(site_info)

{'sites': [Site(name='urn:hpflex/s223-test_site#s223-test_site', noaastations=1, latitudes=1, longitudes=1, timezones=1)], 'zones': [Hvac-zone(name='urn:hpflex/s223-test_site#zone_3', hvacs=1, tstats=1, windows=1, spaces=1), Hvac-zone(name='urn:hpflex/s223-test_site#zone_2', hvacs=1, tstats=1, windows=1, spaces=1), Hvac-zone(name='urn:hpflex/s223-test_site#zone_1', hvacs=1, tstats=1, windows=2, spaces=2)]}


In [ ]:
zone = site_info['zones'][0]
print(zone)

Hvac-zone(name='urn:hpflex/s223-test_site#zone_3', hvacs=1, tstats=1, windows=1, spaces=1)


In [ ]:
zone.spaces

[Space(name='urn:hpflex/s223-test_site#space_3_1', area=Value(value=4.0, unit='http://qudt.org/vocab/unit/FT2'))]

In [ ]:
zone.windows[0].area.name

rdflib.term.URIRef('urn:hpflex/s223-test_site#window_3_3-area-name')

In [ ]:
zone.tstats[0]

Tstat(name='urn:hpflex/s223-test_site#tstat_zone_3', stage_count=Value(value=3.0, unit='None'), resolution=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), setpoint_deadband=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tolerance=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), active=Value(value=3.0, unit='None'))

In [ ]:
print(zone.tstats[0].resolution)
zone.tstats[0].resolution.convert_to_si()
print(zone.tstats[0].resolution)
print(zone.tstats[0].resolution.is_delta)

Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F')
Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')
True


In [ ]:
# optionally just load everything as si 
si_loader = LoadModel(f"{ontology}-test_site/test_build/test_build.ttl", ontology = 'brick', as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].resolution)

Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')


In [ ]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

In [ ]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

In [ ]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [ ]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [ ]:
# bg.create_dashboard('AQL from brick')

In [ ]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')